In [1]:
import tensorflow as tf
import numpy as np

In [2]:
### Hyperparameters
S = 3 # *6*6 number of nodes in softmax layer
layer_shapes = [(1,5),(5,10,),(10,20),(20,S)]
strides = [1,2,1,2]

batch_size = 10

In [3]:
### Some helper functions to tidy things up

#Define the variables in my network
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.2, shape=shape,dtype=tf.float32)
    return tf.Variable(initial)

#Some convolutions
def conv2d(x, W,stride):
    if stride == 2:
        pad = "VALID"
    else:
        pad = "SAME"
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1],padding =  pad)

def conv(im,W,b,stride,shift,scale):
    with tf.name_scope('conv'):
        o = conv2d(im, W,stride) + b #convolve
    with tf.name_scope('batch_norm'):
        mean,var = tf.nn.moments(o,axes = [0]) #batch normalise
        x = scale*(o-mean)/tf.sqrt(tf.square(var)+1e-7) + shift
    with tf.name_scope('activation'):
        return tf.nn.relu(x) #activation function

In [4]:
#A layer in my net
class Conv_layer():
    def __init__(self,shapes,name,w=3,stride = 2,dropout = False):
        K1,K2 = shapes
        self.stride = stride
        self.window = w
        self.dropout = dropout
        
        with tf.name_scope('params'+name):
            self.weights = weight_variable([w,w,K1,K2])
            self.biases = bias_variable([K2])
            self.shift = tf.Variable(0.0)
            self.scale = tf.Variable(1.0)
        
    def forward(self,x):
        with tf.name_scope('layer'):
            if self.dropout:
                return tf.nn.dropout(conv(x,self.weights,self.biases,self.stride,self.shift,self.scale), keep_prob)
            else:
                return conv(x,self.weights,self.biases,self.stride,self.shift,self.scale)

In [5]:
# Define inputs and outputs
with tf.name_scope('inputs'):
    image = tf.placeholder(tf.float32, shape=[None, 28,28,1])
    lab = tf.placeholder(tf.int64, shape=[None, 1])
    y_ = tf.one_hot(lab,10,0.0,1.0)
    y_ = tf.reshape(y_,[batch_size,10])
    keep_prob = tf.placeholder(tf.float32)

### Generate the layers for our network
net = []
for i,shape in enumerate(layer_shapes):
    net.append(Conv_layer(shape,str(i),stride = strides[i]))

with tf.name_scope('softparams'):
    W_soft = weight_variable([S*6*6, 10])
    b_soft = bias_variable([10])
    
### Connect the computational graph
n=len(net)
outputs = (n+1)*[None]
outputs[0] = image
for i in range(n):
    outputs[i+1] = net[i].forward(outputs[i])
    
with tf.name_scope('softmax'):
    y_conv = tf.nn.softmax(tf.matmul(tf.reshape(outputs[n],[batch_size,S*6*6]),W_soft) + b_soft)

In [6]:
### Loss and training
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
    tf.scalar_summary('cross_entropy',cross_entropy)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.scalar_summary('accuracy',accuracy)
    
with tf.name_scope('trainer'):
    train_step = tf.train.RMSPropOptimizer(0.01, momentum = 0.5).minimize(cross_entropy)

In [7]:
# def update_rule(loss,param,eta):
#     with tf.name_scope('grads'):
#         grads = tf.gradients(loss,param)
    
# eta = []

# with tf.name_scope('optimiser'):
#     for param in parameters:
        

In [8]:
### Get some data
from fuel.datasets import MNIST
mnist = MNIST(('train',))
state = mnist.open()

In [9]:
summaries_dir = '/Volumes/ExtraDiskSpace/Documents/CS231n/TF/grads'

if tf.gfile.Exists(summaries_dir):
    tf.gfile.DeleteRecursively(summaries_dir)
tf.gfile.MakeDirs(summaries_dir)

merged = tf.merge_all_summaries()

In [10]:
count = 0
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    writer = tf.train.SummaryWriter(summaries_dir,sess.graph)
    
    
    requests = [[np.random.randint(1,5000) for n in range(batch_size)] for epoch in range(10)]
    for req in requests:
        count +=1
        im,labels = mnist.get_data(state=state, request=req)
        im = im.reshape((batch_size,28,28,1))/255
        feed = {image: im, lab: labels, keep_prob: 1.0}

        _,acc,loss,y,x = sess.run([train_step,accuracy,cross_entropy,y_conv,outputs[1]], feed_dict= feed)
        print(acc,loss)

        summary = sess.run(merged,feed_dict=feed)
        writer.add_summary(summary,count)

        #G = sess.run(grads, feed_dict= feed)
    

0.0 207.201
0.1 207.452
0.0 207.138
0.1 207.123
0.2 207.4
0.1 207.098
0.0 207.75
0.0 207.18
0.0 207.417
0.0 207.069
